# Install and import

In [ ]:
!pip install -U datasets

In [ ]:
import torch, numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import transformers
from peft import PeftModel
import matplotlib.pyplot as plt
import random
from datasets import Dataset
from IPython.display import HTML, display

In [ ]:
!pip install -U inseq

In [ ]:
from inseq import load_model

# Explainability

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
BASE_MODEL   = "sshleifer/distilbart-cnn-12-6"
ADAPTER_DIR  = "/content/drive/MyDrive/distilbart_wikihow_headline_lora"

tok   = AutoTokenizer.from_pretrained(ADAPTER_DIR)
base  = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL, device_map={"":0})
model = PeftModel.from_pretrained(base, ADAPTER_DIR)
model.eval()

ins_model = load_model(ADAPTER_DIR, "integrated_gradients")
ins_model.to(device)

/usr/local/lib/python3.11/dist-packages/peft/config.py:165: UserWarning: Unexpected keyword arguments ['qalora_group_size', 'use_qalora'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


HuggingfaceEncoderDecoderModel(
  (model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
        (layers): ModuleList(
          (0-11): 12 x BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1024, bias=False)
                )
                (lora_embe

In [ ]:
df   = Dataset.from_csv("/content/drive/MyDrive/wikihow_clean.csv").to_pandas()
data = Dataset.from_pandas(df).shuffle(seed=SEED)

tmp        = data.train_test_split(test_size=0.20, seed=SEED)
train_full = tmp["train"]
temp       = tmp["test"].train_test_split(test_size=0.50, seed=SEED)
val, test  = temp["train"], temp["test"]

In [ ]:
sample_idx = 0
article = test[sample_idx]["text"]

inputs = tok(article, return_tensors="pt",
             truncation=True, max_length=1024).to(device)
summary_ids = model.generate(**inputs,
                             max_length=64,
                             num_beams=4,
                             early_stopping=True)
generated = tok.batch_decode(summary_ids,
                             skip_special_tokens=True)[0]

print("=== ARTICLE ===\n", article[:500], "...\n")
print("=== SUMMARY ===\n", generated)

=== ARTICLE ===
 New York City is a great place to visit on your own. To plan a fun-filled solo trip, start by using a solid map or perhaps even finding a tour guide. ...

=== SUMMARY ===
 Visit New York City on a Solo Trip to the U.S. Without a Tour Guide to the Biggest City in the United States (for Solo Touring) City of New York (Solo Touring the City of the Bigest Places in the World


In [ ]:
DECODE_CONFIG = {
    "greedy": dict(num_beams=1, do_sample=False),

    "beam":   dict(num_beams=4, do_sample=False),

    "top_k":  dict(num_beams=1, do_sample=True,  top_k=50,  top_p=1.0, temperature=1.0),

    "top_p":  dict(num_beams=1, do_sample=True,  top_k=0,   top_p=0.92, temperature=1.0),
}


In [ ]:
def generate_and_attribute(article: str,
                           decode_mode: str,
                           attr_method: str = "integrated_gradients"):
    gen_kwargs = DECODE_CONFIG[decode_mode].copy()
    tok_in = tok(article, return_tensors="pt",
                 truncation=True, max_length=1024).to(device)

    with torch.no_grad():
        summary_ids = model.generate(**tok_in,
                                     max_length=64,
                                     **gen_kwargs)

    summary = tok.batch_decode(summary_ids,
                               skip_special_tokens=True)[0]

    at = ins_model.attribute(article, summary,
                             method=attr_method,
                             n_steps=32)

    flat_scores = {}
    src_attr = at.get_scores_dicts()[0]["source_attributions"]
    for _, src_dict in src_attr.items():
        for src_tok, val in src_dict.items():
            flat_scores[src_tok] = flat_scores.get(src_tok, 0.0) + val

    return summary, at, flat_scores

In [ ]:
attrs_html = []
summaries  = {}

for mode in DECODE_CONFIG:
    summ, attr, _ = generate_and_attribute(article, mode)
    summaries[mode] = summ

    heatmap_html = attr.show(display=False, return_html=True)
    header = f"<h3 style='margin-top:2em'>{mode.upper()} — {summ}</h3>"
    attrs_html.append(header + heatmap_html)

display(HTML("".join(attrs_html)))

The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Attributing with integrated_gradients...: 100%|██████████| 55/55 [00:24<00:00,  2.21it/s]
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Attributing with integrated_gradients...: 100%|██████████| 56/56 [00:27<00:00,  1.97it/s]
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Attributing with integrated_gradients...: 100%|██████████| 56/56 [00:26<00:00,  2.06it/s]


,<s>,Visit,ĠNew,ĠYork,ĠCity,ĠSolo,Ġon,Ġa,ĠSolo,ĠTrip,Ġto,ĠNew,ĠYork,Ġ,Ġ,Ġ,Ġ(,for,Ġ$,100,",",000,),Ġin,ĠNew,ĠYork,ĠState,Ġof,Ġthe,ĠCity,'s,ĠFirst,ĠLady,Ġ(,for,Ġthe,ĠU,.,S,.,ĠA,.,ĠNew,ĠYork,),ĠTour,ist,ĠTour,ist,ĠGuide,ĠGuide,ĠGuide,Ġto,</s>
<s>,0.013,0.022,0.019,0.014,0.011,0.041,0.013,0.023,0.024,0.019,0.008,0.017,0.014,0.012,0.011,0.013,0.013,0.016,0.026,0.016,0.01,0.01,0.009,0.014,0.013,0.012,0.008,0.014,0.041,0.014,0.009,0.011,0.016,0.012,0.012,0.01,0.015,0.012,0.018,0.012,0.012,0.008,0.024,0.013,0.011,0.015,0.015,0.014,0.013,0.012,0.008,0.008,0.011,0.016
New,0.022,0.027,0.067,0.059,0.033,0.017,0.027,0.015,0.017,0.019,0.019,0.081,0.037,0.019,0.021,0.024,0.028,0.038,0.027,0.04,0.024,0.02,0.021,0.036,0.025,0.022,0.026,0.027,0.028,0.028,0.03,0.029,0.032,0.019,0.029,0.025,0.029,0.054,0.043,0.027,0.032,0.038,0.041,0.02,0.027,0.024,0.019,0.017,0.019,0.017,0.02,0.021,0.02,0.043
ĠYork,0.013,0.038,0.036,0.086,0.044,0.027,0.014,0.014,0.023,0.024,0.017,0.073,0.042,0.011,0.012,0.014,0.021,0.044,0.025,0.02,0.014,0.01,0.012,0.02,0.027,0.021,0.031,0.038,0.02,0.047,0.025,0.022,0.023,0.013,0.016,0.019,0.02,0.042,0.031,0.02,0.025,0.017,0.042,0.012,0.019,0.023,0.018,0.012,0.013,0.011,0.015,0.016,0.019,0.035
ĠCity,0.013,0.032,0.02,0.038,0.092,0.023,0.013,0.013,0.02,0.021,0.016,0.054,0.033,0.012,0.013,0.014,0.018,0.029,0.02,0.016,0.014,0.011,0.013,0.021,0.015,0.016,0.045,0.077,0.021,0.049,0.04,0.032,0.025,0.013,0.023,0.016,0.023,0.029,0.026,0.019,0.028,0.018,0.038,0.011,0.024,0.033,0.018,0.014,0.014,0.012,0.018,0.019,0.019,0.032
Ġis,0.012,0.012,0.014,0.023,0.029,0.009,0.011,0.012,0.009,0.009,0.011,0.017,0.016,0.011,0.014,0.016,0.014,0.014,0.014,0.013,0.011,0.012,0.01,0.017,0.011,0.012,0.015,0.018,0.015,0.012,0.02,0.013,0.024,0.011,0.022,0.012,0.013,0.012,0.016,0.012,0.02,0.019,0.021,0.011,0.017,0.013,0.009,0.009,0.01,0.009,0.014,0.015,0.014,0.021
Ġa,0.018,0.008,0.01,0.015,0.015,0.008,0.012,0.009,0.006,0.007,0.009,0.01,0.013,0.009,0.01,0.012,0.011,0.016,0.011,0.009,0.01,0.01,0.009,0.013,0.012,0.011,0.013,0.023,0.014,0.011,0.013,0.01,0.018,0.01,0.011,0.011,0.015,0.01,0.011,0.01,0.013,0.011,0.029,0.01,0.013,0.009,0.007,0.009,0.009,0.009,0.014,0.013,0.013,0.018
Ġgreat,0.016,0.018,0.015,0.019,0.026,0.012,0.015,0.011,0.01,0.011,0.015,0.015,0.016,0.011,0.015,0.017,0.018,0.017,0.018,0.014,0.015,0.013,0.012,0.018,0.014,0.013,0.014,0.032,0.017,0.012,0.025,0.015,0.023,0.014,0.016,0.013,0.023,0.019,0.018,0.021,0.013,0.02,0.027,0.013,0.016,0.014,0.012,0.012,0.012,0.01,0.019,0.018,0.019,0.021
Ġplace,0.021,0.025,0.018,0.03,0.039,0.016,0.022,0.018,0.016,0.013,0.022,0.02,0.019,0.015,0.02,0.022,0.023,0.016,0.023,0.017,0.017,0.016,0.023,0.022,0.019,0.016,0.016,0.031,0.032,0.015,0.029,0.03,0.031,0.016,0.027,0.022,0.028,0.025,0.025,0.035,0.018,0.034,0.026,0.016,0.02,0.023,0.016,0.016,0.021,0.013,0.021,0.022,0.024,0.027
Ġto,0.023,0.021,0.02,0.035,0.039,0.018,0.027,0.033,0.033,0.011,0.021,0.017,0.031,0.016,0.022,0.021,0.016,0.029,0.028,0.017,0.021,0.021,0.03,0.013,0.022,0.019,0.012,0.033,0.019,0.019,0.032,0.019,0.021,0.013,0.021,0.019,0.038,0.039,0.041,0.042,0.019,0.026,0.028,0.018,0.014,0.029,0.011,0.014,0.014,0.013,0.019,0.021,0.025,0.02
Ġvisit,0.028,0.063,0.028,0.032,0.03,0.039,0.042,0.035,0.031,0.023,0.019,0.022,0.05,0.028,0.031,0.027,0.023,0.027,0.03,0.021,0.029,0.031,0.031,0.017,0.029,0.033,0.018,0.029,0.022,0.028,0.018,0.019,0.019,0.019,0.019,0.019,0.034,0.033,0.039,0.054,0.018,0.021,0.031,0.031,0.021,0.026,0.026,0.022,0.02,0.024,0.021,0.022,0.023,0.021
